In [1]:
import re
import time

import pandas as pd
import requests
from requests_html import HTML

Except for the number of pages to scrape and the size of them. The code is pretty much ready, we need to polish 
it and write the narrative around the examples. 

TODO:

    - Write intro
    - Recap futures api and delayed shortly
    - Introduce the "SO problem"
    - Write text around how to solve it serialy
    - Write text around multiple ways to parallelize the workflow
    - Write text around as_completed, vs dask dataframe example
    - Select some parts to be exercises. 
     

Note:
Throttling limitation
https://api.stackexchange.com/docs/throttle

If an application does not have an access_token, then the application shares an IP based quota with all other applications on that IP. This quota is based on the key being passed by the applications; it is the max of the daily request limit for the applications involved, which by default is 10,000. This quota scheme is essentially unchanged from earlier versions of the API.

In [4]:
# r = requests.get(url)
# html_str = r.text
# html = HTML(html=html_str)

`.s-post-summary`: Base parent container for a post summary

https://stackoverflow.design/product/components/post-summary/

In [5]:
# @dask.delayed
def clean_scraped_data(text, keyname=None):
    """
    Cleans the scraped data once in text format
    """
    if keyname == "stats":
        text_strings = re.findall(r"\d+", text)
        
        try:
            text =[int(s) for s in text_strings]
        except ValueError:
            text =[int(re.sub("k", "000", s)) for s in text_strings]

    elif keyname == "tags":
        return text.split("\n")
    return text

In [6]:
# @dask.delayed
def extract_url_and_parse_html(url):
    """
    Given a SO url gets questions summary and cleans data.
    
    see also: clean_scraped_data
    """
    # function that will parse a single page
    r = requests.get(url)
    
    if r.status_code not in range(200, 299):
        return []
    
    #get html
    html = HTML(html=r.text)
        
    questions = html.find(".s-post-summary")
    
    key_class_dict = {"title": ".s-post-summary--content-title" ,
                  "stats": ".s-post-summary--stats",
                  "tags": ".s-post-summary--meta-tags"}

    datas = []

    for q_el in questions:
        q_data = {}
        for k, v in key_class_dict.items():
            q_data[k] = clean_scraped_data(q_el.find(v, first=True).text, k)
        
        q_data["vote"] = q_data["stats"][0]
        q_data["answer"] = q_data["stats"][1]
        q_data["view"] = q_data["stats"][2]
        datas.append(q_data)
    return datas

In [11]:
# def scrape_tag(tag="python", query_filter="Newest", max_pages=100, pagesize=50):
#     base_url = "https://stackoverflow.com/questions/tagged/"
#     datas_page = []
#     for p in range(max_pages):
#         page_num = p + 1
#         url = f"{base_url}{tag}?tab={query_filter}&page={page_num}&pagesize={pagesize}"
#         datas_page += extract_url_and_parse_html(url)
#         #time.sleep(1.2) #need between requests.
#     return datas_page

In [7]:
def scrape_tag(page_num, tag="python", query_filter="Newest", pagesize=50):
    base_url = "https://stackoverflow.com/questions/tagged/"
    url = f"{base_url}{tag}?tab={query_filter}&page={page_num}&pagesize={pagesize}"
    datas_page = extract_url_and_parse_html(url)
        #time.sleep(1.2) #need between requests.
    return pd.DataFrame(datas_page)

## Sequential

In [9]:
%%time
pd_res = []
for page in range(1, 11):
    pd_res.append(scrape_tag(page))

##pd_res

CPU times: user 2.2 s, sys: 36.6 ms, total: 2.24 s
Wall time: 3.94 s


## client.submit

### NOTE: ADD SOMETHING WITH wait. 

In [11]:
from dask.distributed import Client


In [12]:
client = Client()
client

2022-12-06 15:36:33,980 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/1y/ydztfpnd11b6qmvbb8_x56jh0000gn/T/dask-worker-space/worker-0dx8gl4i', purging
2022-12-06 15:36:33,980 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/1y/ydztfpnd11b6qmvbb8_x56jh0000gn/T/dask-worker-space/worker-7ddj6qvo', purging
2022-12-06 15:36:33,981 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/1y/ydztfpnd11b6qmvbb8_x56jh0000gn/T/dask-worker-space/worker-2dq1v_qk', purging
2022-12-06 15:36:33,982 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/1y/ydztfpnd11b6qmvbb8_x56jh0000gn/T/dask-worker-space/worker-m1xmp1he', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:61533,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:61548,Total threads: 2
Dashboard: http://127.0.0.1:61554/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:61539,


In [14]:
%%time
fut = []
for page_num in range(1, 11):
    future = client.submit(scrape_tag, page_num)
    fut.append(future)

res = client.gather(fut)

CPU times: user 209 ms, sys: 77.1 ms, total: 286 ms
Wall time: 1.46 s


In [15]:
fut[0]

<Future: finished, type: pandas.core.frame.DataFrame, key: scrape_tag-e83a617288abc03472fe398fc09cd480>

In [18]:
res[0].head(3)

,title,stats,tags,vote,answer,view
0,Python errors in command prompt,"[0, 0, 4]","[python, python-3.x, python-2.7]",0,0,4
1,How to use KinematicTrajectoryOptimization in ...,"[1, 0, 3]","[python, optimization, drake]",1,0,3
2,Python Selenium & Browserstack - Connect to proxy,"[0, 0, 6]","[python, selenium, browserstack]",0,0,6


### client.map
lambda version or none lambda version

In [21]:
del fut

In [28]:
%%time
t = client.map(scrape_tag, range(1, 11))
test = client.gather(t)
#wait(t)

CPU times: user 76 ms, sys: 15.4 ms, total: 91.4 ms
Wall time: 831 ms


In [30]:
test[0].head(3)

,title,stats,tags,vote,answer,view
0,Python errors in command prompt,"[0, 0, 4]","[python, python-3.x, python-2.7]",0,0,4
1,How to use KinematicTrajectoryOptimization in ...,"[1, 0, 3]","[python, optimization, drake]",1,0,3
2,Python Selenium & Browserstack - Connect to proxy,"[0, 0, 6]","[python, selenium, browserstack]",0,0,6


In [31]:
%%time
t_d = client.map(lambda p: scrape_tag(p, tag="dask"), range(1, 11))
t_d_res = client.gather(t_d)

CPU times: user 104 ms, sys: 31.6 ms, total: 136 ms
Wall time: 1.23 s


In [33]:
t_d_res[0].head(3)

,title,stats,tags,vote,answer,view
0,Load data with chunks,"[0, 1, 20]","[python, pandas, sqlalchemy, dask, chunks]",0,1,20
1,Apply a function over the columns of a Dask array,"[2, 0, 23]","[dask, dask-distributed, dask-dataframe, dask-...",2,0,23
2,Is py-datatable compatible with Dask?,"[0, 0, 18]","[dask, py-datatable]",0,0,18


## use delayed

In [28]:
import dask

In [39]:
@dask.delayed
def scrape_tag_delayed(page_num, tag="python", query_filter="Newest", pagesize=50):
    base_url = "https://stackoverflow.com/questions/tagged/"
    url = f"{base_url}{tag}?tab={query_filter}&page={page_num}&pagesize={pagesize}"
    datas_page = extract_url_and_parse_html(url)
        #time.sleep(1.2) #need between requests.
    return pd.DataFrame(datas_page)

In [40]:
res = []
for page in range(1, 11):
    res.append(scrape_tag_delayed(page))

In [41]:
res

[Delayed('scrape_tag_delayed-2cc531c7-c50d-40af-bf65-d3a1909de259'),
 Delayed('scrape_tag_delayed-7980fbe0-dbc5-46f6-b79b-379617bead4b'),
 Delayed('scrape_tag_delayed-ca4684a0-a78d-4241-ab0f-c0bea51900e5'),
 Delayed('scrape_tag_delayed-e8a95c0e-e2ad-432e-929a-c4267b7fda68'),
 Delayed('scrape_tag_delayed-6df82447-fc5f-49f9-8f8a-84f2addade57'),
 Delayed('scrape_tag_delayed-fce202db-e24c-42c0-99bf-ca28040f48fc'),
 Delayed('scrape_tag_delayed-812e0f2d-558b-4262-a56c-fa5a6c05680a'),
 Delayed('scrape_tag_delayed-18e7de00-63e9-471b-a921-dbb9c7080fe2'),
 Delayed('scrape_tag_delayed-2c163ab4-4732-49a1-b093-b443499c3428'),
 Delayed('scrape_tag_delayed-b17adc02-e4c2-42a9-909f-b3f0d6a16c6f')]

In [33]:
%%time
r = dask.compute(res)

CPU times: user 99.9 ms, sys: 30.3 ms, total: 130 ms
Wall time: 1.21 s


In [169]:
from dask.distributed import as_completed

## NOTE
HERE DO EXAMPLE BELOW WITH WAIT AND TIME IT 

In [189]:
%%time
t15 = client.map(scrape_tag, range(15))

tot_views= 0 

for future in as_completed(t15):
    test = future.result()['view']
    tot_views += test.sum()

tot_views

CPU times: user 35.5 ms, sys: 7.28 ms, total: 42.8 ms
Wall time: 51.7 ms


105874

In [118]:
import dask.dataframe as dd

In [183]:
test_del = dd.from_delayed(t15)

In [184]:
test_del

,title,stats,tags,vote,answer,view
npartitions=15,,,,,,
,object,object,object,int64,int64,int64
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [185]:
test_del['view'].sum()

dd.Scalar<series-..., dtype=int64>

In [186]:
%%time
test_del['view'].sum().compute()

CPU times: user 37.5 ms, sys: 4.54 ms, total: 42.1 ms
Wall time: 49 ms


105874

In [191]:
client.shutdown()